<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/086_AgentZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ Agent System Components (with Planning & Reflection)

1. **Tool** – Function that does something useful (e.g., `create_plan`, `track_progress`, domain tools)
2. **Tool Registry** – Stores all tools by name for lookup by the agent
3. **ActionContext** – Holds dependencies (e.g., memory, auth, services, registry)
4. **Environment** – Executes tools using the ActionContext for dependency injection
5. **Agent** – Parses input, chooses tools, calls them via Environment
6. **Capabilities** – Modular behaviors that hook into the agent loop (e.g., **PlanFirstCapability**, **ProgressTrackingCapability**)
7. **Dependencies** – Injected values (e.g., SMTP, tokens, LLM) automatically provided to tools
8. **Wiring** – Instantiate and connect everything together (Agent + Environment + Capabilities + Tools)



For a **Daily Goal Tracker Agent**, the **first question** we ask is:

> *“What exactly do I want the agent to do for me, from start to finish, if I were explaining it to a friend?”*

Here’s a super-simple first answer:

1. **Ask me what my goals are for today** (just 2–3 things).
2. **Write a small plan** for completing them.
3. **Record my progress** on at least the first goal.
4. **Tell me what’s been done so far** and what’s left.

---

Once we have that, we can break it into **steps the agent would follow**:

1. **Get today’s goals** from me.
2. **Make a plan** — outline the steps.
3. **Update progress** — mark one as “in progress” or “done”.
4. **Summarize** — output the plan and progress.





## 🎯 Daily Goal Tracker — 4 Tiny Tools

1. **📝 set\_goals**
   *Ask me for today’s goals (2–3 items) and remember them.*

   * **Input:** list of short goal strings
   * **Does:** stores them in `memory["goals"]`
   * **Output:** confirms the saved goals

2. **📋 create\_plan**
   *Turn today’s goals into a quick 3-step plan.*

   * **Input:** none (reads from memory)
   * **Does:** writes `state["current_plan"]` with steps
   * **Output:** the plan list

3. **✅ update\_progress**
   *Log progress on one of the steps.*

   * **Input:** `step_index`, `status` (`"in_progress"` / `"done"`), optional `note`
   * **Does:** appends an entry to `memory["progress"]`
   * **Output:** the logged entry

4. **📊 summarize\_progress**
   *Give a quick status report.*

   * **Input:** none
   * **Does:** checks plan + progress to see what’s done and what’s left
   * **Output:** a short, readable summary

---

### 🛠 How They Fit the Framework

* **Tool Registry:** holds all four tools
* **ActionContext / Memory:** keeps `goals`, `plan`, `progress`
* **Environment:** runs tools, injects any needed dependencies
* **Capabilities (optional):**

  * *PlanFirst* — make sure we plan before acting
  * *ProgressTracking* — automatically mirror updates (can add later)






# 1) Tools

## What to learn (the essentials)

* **Clear contract**
  Know the *inputs* and the exact *JSON* your tool returns. Tiny, stable shapes win:
  `{"ok": True, "plan": [...]}` or `{"ok": False, "error": "why"}`.

* **Single responsibility**
  Each tool does *one* thing. If you describe it with “and then…”, it’s two tools.

* **Stateless behavior**
  No hidden globals. Anything shared lives in **ActionContext.memory** (per run) or **ActionContext.deps** (injected services).

* **Where side-effects go**

  * Tools may write to `ActionContext.memory` (e.g., progress log).
  * The **agent loop** writes to its own `state` (e.g., `state["current_plan"]`) from tool *outputs*.
    Keep that split in mind.

* **Validation + normalization (lightweight)**
  Check inputs are well-formed (e.g., list of non-empty strings), trim whitespace, cap lengths, and return friendly errors.

* **Deterministic & testable**
  Avoid randomness or time unless injected via `ActionContext.deps["clock"]`. Makes unit tests trivial.

* **Tiny docstring**
  Add a one-liner: **Purpose / Reads / Writes / Returns**. Future-you will thank present-you.

---

## Mini rubric (apply to each tool)

* Does it have a **clear input → output**?
* Is it **one job** only?
* Does it avoid touching **agent `state`** directly?
* If it writes memory, is it **append or set** (and why)?
* Are inputs **validated** (type, allowed values)?
* Would a unit test be **2–3 lines**?

In [ ]:
# --- Tools ---

def set_goals(action_context: ActionContext, goals: list[str]) -> dict:
    # would write: action_context.memory["goals"] = list(goals)
    return {"goals": goals}

def create_plan(action_context: ActionContext) -> dict:
    # would read: action_context.memory["goals"]
    # agent loop will store into: state["current_plan"]
    return {"plan": ["Step 1 ...", "Step 2 ...", "Step 3 ..."]}

def update_progress(action_context: ActionContext,
                    step_index: int,
                    status: str,
                    note: str = "") -> dict:
    # would append to: action_context.memory["progress"]
    return {"progress_entry": {"step": step_index, "status": status, "note": note}}

def summarize_progress(action_context: ActionContext) -> dict:
    # would read: state["current_plan"] + action_context.memory["progress"]
    return {"summary": "1 in progress, 2 remaining..."}


## What is “state”? What is “stateless behavior”?

Think of your agent run like a short project.
**State** is anything the system *remembers* from one step to the next during that project.

## Two kinds of state we use (on purpose)

* **Agent state** (the agent’s whiteboard)

  * Owned by the **agent loop**.
  * Examples: `state["current_plan"]`, flags like `did_progress`.
  * Tools **don’t** write here directly; the agent copies tool outputs into it.

* **ActionContext.memory** (the tools’ shared backpack)

  * Owned by **tools** for the duration of the run.
  * Examples: `memory["goals"]`, `memory["progress"]`.
  * Tools read/write this to pass info to each other.

*(Later, you might add persistent state—files/DB—across runs. Not needed now.)*

---

## “Stateless behavior” (what we want in tools)

A **stateless** tool:

* Depends only on its **inputs** and the **ActionContext** it’s given.
* Makes no hidden assumptions (no globals, no singletons).
* Has **predictable** output: same inputs → same outputs.
* If it needs time/randomness, it gets them from **injected deps** (e.g., `ActionContext.deps["clock"]`) so tests can control them.

### Tiny examples

**✅ Good (stateless):**

```python
def update_progress(action_context, step_index, status, note=""):
    # reads nothing hidden; writes to shared backpack
    entry = {"step": step_index, "status": status, "note": note}
    action_context.memory.setdefault("progress", []).append(entry)
    return {"ok": True, "entry": entry}
```

**🚫 Not great (stateful in a hidden way):**

```python
GLOBAL_PROGRESS = []  # hidden global — hard to test/reuse

def update_progress(action_context, step_index, status, note=""):
    GLOBAL_PROGRESS.append({...})  # writes outside the backpack
    return {"ok": True}
```

**✅ Good with injected time:**

```python
def update_progress(action_context, step_index, status, note=""):
    clock = action_context.deps.get("clock")  # injectable
    ts = clock.time() if clock else None
    entry = {"step": step_index, "status": status, "note": note, "ts": ts}
    action_context.memory.setdefault("progress", []).append(entry)
    return {"ok": True, "entry": entry}
```

---

## Quick checklist for your tools

* **Inputs/Outputs clear?** Small JSON like `{"ok": True, ...}`
* **One job only?** If you say “and then…”, split it.
* **No globals?** Only use `ActionContext` for shared data/deps.
* **Agent state untouched?** Let the **agent** copy tool outputs into `state`.
* **Deterministic?** Same inputs → same outputs (or controlled via injected deps).





## Why globals are trouble vs. ActionContext

* **Scope & lifetime**

  * **Global:** lives for the whole Python process. Every run, every test, every agent shares it.
  * **ActionContext.memory:** created fresh for *this* agent run. Throw it away when done.

* **Isolation**

  * **Global:** data leaks between runs (“yesterday’s progress” shows up today).
  * **ActionContext:** each run gets its own backpack → no cross-contamination.

* **Testing**

  * **Global:** tests must remember to reset/cleanup; flaky and order-dependent.
  * **ActionContext:** pass a fresh context per test → deterministic, independent tests.

* **Concurrency**

  * **Global:** two agents updating it at once can clobber each other.
  * **ActionContext:** each agent updates its own memory → no collisions.

* **Explicitness & DI**

  * **Global:** hidden dependency; tools magically reach out to a module variable.
  * **ActionContext:** dependency is **explicitly** passed in; tools only touch what you give them.

* **Swappability**

  * **Global:** hard-wired. Want to switch to a DB later? Lots of refactors.
  * **ActionContext:** change what’s in the backpack (e.g., swap `deps["clock"]`) without touching tool code.

## Tiny contrast

### With a global (leaks across runs)

```python
GLOBAL_PROGRESS = []  # 😬 shared forever

def update_progress(step, status):
    GLOBAL_PROGRESS.append({"step": step, "status": status})
    return {"ok": True}

# Run A
update_progress(0, "in_progress")
# Run B (later) accidentally starts with leftover data from Run A
print(GLOBAL_PROGRESS)  # [{'step': 0, 'status': 'in_progress'}]
```

### With ActionContext (scoped per run)

```python
class ActionContext:
    def __init__(self):
        self.memory = {}

def update_progress(ctx: ActionContext, step, status):
    log = ctx.memory.setdefault("progress", [])
    log.append({"step": step, "status": status})
    return {"ok": True}

# Run A
ctx_a = ActionContext()
update_progress(ctx_a, 0, "in_progress")
print(ctx_a.memory["progress"])  # [{'step': 0, 'status': 'in_progress'}]

# Run B (clean)
ctx_b = ActionContext()
print(ctx_b.memory.get("progress"))  # None  ✅ no leakage
```

## Mental model

* **Global variable:** a messy, shared whiteboard in the hallway—everyone writes on it.
* **ActionContext:** your *own* notebook for this session—clean, portable, easy to archive.

**TL;DR:** Globals are shared, implicit, and leaky; ActionContext is explicit, per-run, and testable. Use the backpack.


# 2) Tool Registry

In [ ]:
# Step 2 — Tool Registry

TOOL_REGISTRY = {
    "set_goals": set_goals,
    "create_plan": create_plan,
    "update_progress": update_progress,
    "summarize_progress": summarize_progress,
}

## What are Decorators?

Short version: **decorators** are just a nicer way to add a function to your registry (and optionally attach metadata) the moment the function is defined. Your **dict registry** does the same thing, but you register tools later, manually.

---

## TL;DR

* **Dict registry:** explicit and simple—good for learning.
* **Decorator registry:** less boilerplate and supports **metadata** (tags, schema, etc.) cleanly.

---

## What changes with a decorator?

### Manual (what you have now)

```python
TOOL_REGISTRY = {}

def set_goals(action_context, goals: list[str]) -> dict:
    return {"goals": goals}

TOOL_REGISTRY["set_goals"] = set_goals  # <-- register later, by hand
```

### With a decorator (auto-register)

```python
TOOL_REGISTRY = {}

def register_tool(name: str | None = None, **meta):
    def deco(func):
        TOOL_REGISTRY[name or func.__name__] = {"fn": func, **meta}
        return func
    return deco

@register_tool(tags=["daily_goals"])    # <-- registers at definition time
def set_goals(action_context, goals: list[str]) -> dict:
    return {"goals": goals}
```

* The decorator **runs when the function is defined** (import time), inserting it into `TOOL_REGISTRY`.
* You can store **metadata** alongside the function (e.g., `tags`, `schema`, `version`), which is handy later for filtering or for structured tool-calling.

---

## Why people like decorators for tools

* **Less repetition:** no separate “add to dict” lines.
* **Keeps code together:** the function and its registration/metadata live in one place.
* **Metadata-friendly:** perfect for things like `tags`, `input schema`, `rate limits`, etc.
* **Discoverability:** you can scan your codebase for `@register_tool` and see all tools.




# Tools + Registry with Decorators
Here’s why the **decorator-style registry** is nicer than the manual dict approach:

* **Auto-registration at definition time**
  No extra “add to dict” lines. Define the tool → it’s registered. Fewer moving parts = fewer mistakes.

* **Function + metadata live together**
  You can attach tags/schemas/rate limits right on the function (`@register_tool(tags=[...])`). Easier to read and maintain.

* **Consistent naming**
  Defaults to the function name, or you can override once (`@register_tool(name="summarize_progress")`). Prevents typos in the registry key.

* **Scales across files**
  Split tools into modules and just `import` them; each tool self-registers on import. Great for larger projects.

* **Discoverability**
  You (or an IDE/search) can scan for `@register_tool` to see exactly what tools exist—no hunting through glue code.

* **Cleaner wiring**
  The registry is created once; adding tools doesn’t require touching the wiring cell/file again.

Quick reality check (minor trade-offs to remember):

* You must **import** the module for registration to happen (import-time side effect).
* If you care about a strict **order**, you’ll need to manage import order or store an explicit `order` field in metadata.

Overall: decorators reduce boilerplate, keep definitions tidy, and make the codebase easier to grow—exactly what teams look for in production agent work.


In [ ]:
TOOL_REGISTRY: dict[str, callable] = {}

def register_tool(name: str | None = None):
    """Decorator: auto-register a function as a tool."""
    def deco(func):
        TOOL_REGISTRY[name or func.__name__] = func
        return func
    return deco

# (optional) helpers
def list_tools() -> list[str]: return sorted(TOOL_REGISTRY.keys())
def get_tool(name: str): return TOOL_REGISTRY[name]


# --- Tools (decorated) ---

@register_tool()  # name defaults to "set_goals"
def set_goals(action_context: ActionContext, goals: list[str]) -> dict:
    # would write: action_context.memory["goals"] = list(goals)
    return {"goals": goals}

@register_tool()  # defaults to "create_plan"
def create_plan(action_context: ActionContext) -> dict:
    # would read: action_context.memory["goals"]
    # agent loop will store into: state["current_plan"]
    return {"plan": ["Step 1 ...", "Step 2 ...", "Step 3 ..."]}

@register_tool()  # defaults to "update_progress"
def update_progress(action_context: ActionContext,
                    step_index: int,
                    status: str,
                    note: str = "") -> dict:
    # would append to: action_context.memory["progress"]
    return {"progress_entry": {"step": step_index, "status": status, "note": note}}

@register_tool(name="summarize_progress")  # explicit name (optional)
def summarize_progress(action_context: ActionContext) -> dict:
    # would read: state["current_plan"] + action_context.memory["progress"]
    return {"summary": "1 in progress, 2 remaining..."}


# 3) Action Context


## What to understand (at a glance)

* **Purpose:** a *per-run backpack* that every tool receives.
* **Scope:** created fresh for each agent run (no leftovers between runs).
* **Compartments:**

  * `memory` → shared scratchpad for tools this run (e.g., `goals`, `progress`)
  * `deps` → injected helpers/services (e.g., `clock`, `smtp`, tokens, clients)
  * `config` → knobs/flags (e.g., `{"read_only": True, "max_iterations": 6}`)

## How you’ll use it (tiny patterns)

* **Read/write shared data**

  ```python
  action_context.memory["goals"] = ["A", "B", "C"]
  goals = action_context.memory.get("goals", [])
  ```
* **Use injected dependencies**

  ```python
  clock = action_context.get_dep("clock")
  ts = clock.time() if clock else None
  ```
* **Respect run settings**

  ```python
  if action_context.config.get("read_only"):
      return {"ok": False, "error": "Read-only mode"}
  ```

## Do / Don’t

* ✅ **Do** keep tools stateless: everything they need comes via `ActionContext` or function args.
* ✅ **Do** treat `memory` as the place tools communicate.
* ✅ **Do** inject time/IO via `deps` so tests can fake them.
* ❌ **Don’t** use globals for shared data.
* ❌ **Don’t** put huge, long-term data here (that’s for real storage later).
* ❌ **Don’t** let tools write the agent’s `state` (the **agent loop** copies tool outputs into `state`).

## What to practice (micro checks)

* Can you **describe** what lives in `memory` vs `deps` vs `config` for your Daily Goal Tracker?
* Can you **add** one dep (e.g., a fake `clock`) and **read** it in a tool?
* Can you **store** and **retrieve** a single value from `memory` (e.g., `last_plan_ts`)?



In [ ]:
# Step 3 — ActionContext
from dataclasses import dataclass, field
from typing import Any, Dict, Optional

@dataclass
class ActionContext:
    """
    Per-run backpack for tools.
    - memory: short-term data tools share this run (e.g., goals, progress)
    - deps: injected services/utilities (e.g., clock, clients, tokens)
    - config: knobs/flags for this run (e.g., max_iterations)
    """
    memory: Dict[str, Any] = field(default_factory=dict)
    deps: Dict[str, Any] = field(default_factory=dict)
    config: Dict[str, Any] = field(default_factory=dict)

    # (optional) tiny helpers
    def get_dep(self, name: str, default: Optional[Any] = None) -> Any:
        return self.deps.get(name, default)

    def remember(self, key: str, value: Any) -> None:
        self.memory[key] = value




## 🛠 Tools vs. 🧩 Capabilities — The Core Difference

### **Tools**

* **What they are:**
  Actions the agent can *explicitly call* to get something done.
  Think of them as **verbs** you hand to the model: *search\_web*, *send\_email*, *set\_goals*, *update\_progress*.
* **When they run:**
  Only when the agent **chooses** to use them in its reasoning loop.
* **What they handle:**
  Concrete, single-purpose work — often interacting with data, services, or memory.
* **Example here:**

  * `set_goals` — store your goals in memory
  * `create_plan` — make a step list
  * `update_progress` — log a change in status
  * `summarize_progress` — report on plan completion

---

### **Capabilities**

* **What they are:**
  Background behaviors or **rules of the road** that wrap around the loop.
  They don’t “do the work” themselves — they **change how the agent uses tools or processes information**.
* **When they run:**
  Automatically at specific hook points (before/after the model thinks, before/after a tool runs, etc.).
* **What they handle:**
  Consistency, guardrails, extra features that don’t need to be consciously chosen by the model.
* **Example here:**

  * `PlanFirstCapability` — forces the first action to be `create_plan` before any other tool
  * `ProgressTrackingCapability` — automatically logs updates when certain tools run

---

## 🔍 How to Choose

* **Use a *Tool*** when:

  * It’s a *thing the agent might or might not need to do*.
  * It’s a discrete action that produces an observable result.
  * The agent should consciously decide when to use it.
  * Example: *check\_weather*, *add\_calendar\_event*, *set\_goals*.

* **Use a *Capability*** when:

  * You want to *change the rules or flow* for **all runs** without the agent deciding each time.
  * It’s meta-behavior — a *habit, safety measure, or policy*.
  * It wraps around tools/model calls instead of being called like a tool.
  * Example: always plan first, always sanitize output, auto-log tool calls.

---

💡 **In our Daily Goal Tracker:**
We use **tools** for the actual work (setting goals, making a plan, logging updates, summarizing)
and **capabilities** for the “style” or “rules” of how that work happens (like forcing planning before acting).




This is where capabilities feel a little “magical” until you peek under the hood.
The magic comes from **hooks** — predefined points in the agent loop where you can insert your own logic.

---

## 🧩 How Capabilities Work

An agent loop is usually something like:

1. **Model Thinks** → "What should I do next?"
2. **Choose a Tool** (or just reply)
3. **Run the Tool**
4. **Store/Update State**
5. **Repeat or Finish**

**Capabilities** hook into one or more of these steps, like little “interceptors”:

| Hook Point                   | Example Capability Behavior                                                    |
| ---------------------------- | ------------------------------------------------------------------------------ |
| **Before Model Thinks**      | Inject context (“Remember: plan first”), rewrite the goal, enforce constraints |
| **After Model Thinks**       | Check if the chosen tool is safe, override the choice if needed                |
| **Before Tool Runs**         | Log the tool call, add extra inputs, block if unsafe                           |
| **After Tool Runs**          | Record progress automatically, clean up the output, trigger another tool       |
| **Before/After Entire Loop** | Start/stop timers, cap the number of iterations                                |

---

### 🔧 How this is implemented

* Capabilities are **classes** with methods like:

  * `before_think(agent_state)`
  * `after_think(agent_state, model_output)`
  * `before_tool(agent_state, tool_name, args)`
  * `after_tool(agent_state, tool_name, result)`
* The agent loop **calls all active capabilities** at each hook point.
* The capability can:

  * **Inspect** the state (read memory, goal, chosen tool, etc.).
  * **Modify** the state (change the chosen tool, add notes, adjust plan).
  * **Block** or **redirect** actions (e.g., force `create_plan` before others).
  * **Trigger** extra actions (e.g., auto-save after every tool).

---

### 🧠 Why this is useful

* **Consistency:** You can make sure certain things *always* happen (plan first, log progress, check safety).
* **Separation of concerns:** You don’t bloat your tools with rules — capabilities are modular and reusable.
* **Guardrails:** They’re great for adding limits, safety checks, or “house style” without touching core logic.

---

💡 A good analogy:
If the **tools** are the *skills* of your agent, **capabilities** are the *habits and reflexes* that guide how those skills are used — always, without thinking.




Now that you’ve got your **tools** list, the natural next step is to decide whether your agent needs **any capabilities** right away, and if so, which ones.

Since we’re keeping the **Daily Goal Tracker** super simple, we’d start with **just one** capability to demonstrate the concept.

---

## 🧩 Choosing Our First Capability

**Candidate:** `PlanFirstCapability`

* **Hook point:** *before\_tool* — runs before the agent executes the next tool.
* **Behavior:** If the agent hasn’t created a plan yet, force the first tool call to be `create_plan`.
* **Why for us:** Ensures the agent always plans before logging progress or summarizing.

---

### Why not more right now?

Capabilities are powerful, but in a beginner build, adding too many will make the loop harder to follow.
Once you see how **one capability** slots into the flow via a hook, we can layer others in (like auto progress tracking or safety checks).

---

### Mini checklist before we code one

1. **Name it** → `PlanFirstCapability`
2. **What hook?** → `before_tool`
3. **What to check?** → Has a plan been made yet?
4. **What to do?** → If no plan, change the tool call to `create_plan`.



# 4) Capability

Here’s a **super minimal, non-running** sketch that shows how a capability uses **hooks** to shape behavior.

---

## 🧩 `PlanFirstCapability` (pseudo-code)

Goal: if no plan exists yet, **force the next tool** to be `create_plan`.

```python


```

* **Hook used:** `before_tool`
* **What it does:** checks state ➜ if no plan ➜ **swaps** the tool call to `create_plan`
* **Why it’s a capability:** it doesn’t “do work” like a tool; it **steers** the loop

---

## 🔁 Tiny agent loop (pseudo) showing the hook

```python
def agent_loop(goal, tools, capabilities):
    state = {"goal": goal}  # agent's working state
    for step in range(4):
        # Model picks something to do next (toy stand-in)
        (tool_name, args) = model_decides_next_action(state)

        # ── HOOK: let capabilities intercept before tool runs
        for cap in capabilities:
            tool_name, args = cap.before_tool(state, tool_name, args)

        # Run the tool (if any), update state
        result = tools[tool_name](args)           # e.g., create_plan(...)
        if tool_name == "create_plan":
            state["current_plan"] = result["plan"]

        # (…other hooks like after_tool could run here…)

        # Stopping rule for demo
        if done(state): break

    return state
```

* See how `before_tool` is called **every time** before a tool executes?
* If `PlanFirstCapability` sees no plan, it **redirects** the first action to `create_plan`.

---

## ✅ Bonus: what an “after\_tool” hook might look like

Here’s an ultra-simple **progress mirroring** capability that runs on the **after\_tool** hook:

```python
class ProgressTrackingCapability:
    # HOOK: runs after a tool executes
    def after_tool(self, state, tool_name, tool_result):
        if tool_name == "update_progress" and tool_result.get("ok", True):
            log = state.setdefault("progress", [])
            log.append(tool_result["entry"])
```

* **Hook used:** `after_tool`
* **What it does:** listens for `update_progress` and appends the entry to a log
* **Why it’s a capability:** it adds a cross-cutting behavior (consistent logging) without modifying the tool itself




In [ ]:
class PlanFirstCapability:
    # HOOK: runs before any tool executes
    def before_tool(self, state, tool, args):
        if "current_plan" not in state and tool != "create_plan":
            # redirect to create_plan with no args (matches tool signature)
            return "create_plan", {}
        return tool, args

    def after_tool(self, state, tool, result):
        pass  # no-op

class ProgressTrackingCapability:
    def before_tool(self, state, tool, args):
        return tool, args  # no-op

    # HOOK: runs after a tool completes
    def after_tool(self, state, tool, result):
        if tool == "update_progress" and result.get("ok", True):
            entry = result.get("entry")
            if entry:
                state.setdefault("progress", []).append(entry)

def update_progress(action_context: ActionContext, step_index: int, status: str, note: str = "") -> dict:
    # would append to: action_context.memory["progress"]
    return {"ok": True, "entry": {"step": step_index, "status": status, "note": note}}




## What to focus on

## 1) Hooks = when your policy runs

* **before\_tool**: chance to *steer* the next action.
* **after\_tool**: chance to *react* to what just happened.
  You’re learning to place logic at the right hook so you don’t cram rules into tools.

## 2) Separation of concerns (clean roles)

* **Tools** do work (verbs).
* **Capabilities** enforce habits/policies via hooks (no business logic).
  PlanFirst **redirects** behavior; ProgressTracking **logs** outcomes. Neither sends emails, fetches web, etc.

## 3) Safe redirection, not execution

* PlanFirst changes **which tool** will run; it doesn’t run the tool itself.
* It returns a new `(tool_name, args)` pair—pure, predictable.

## 4) Stable, tiny interfaces

* Capabilities assume a **simple result shape** (e.g., `{"ok": True, "entry": {...}}`).
* If that shape changes, only the capability needs a small tweak—not your whole loop.

## 5) State vs. ActionContext (again)

* Capabilities read/write the **agent’s state** (e.g., `state["current_plan"]`, `state["progress"]`).
* Tools read/write **ActionContext.memory**. Keeping this split clear is a core skill.

## 6) Order matters (a little)

* You’re learning that capability order can matter at a hook:

  * Put **PlanFirst** first in `before_tool`, so it can redirect early.
  * **ProgressTracking** lives in `after_tool`, so order there rarely matters.

## 7) Idempotence & guardrails thinking

* PlanFirst only redirects if no plan exists → no loops.
* ProgressTracking checks `ok` and an `entry` exists before logging → no junk.
  This is the habit of making hook logic **safe to run more than once**.

# Tiny checklist (use while reading your code)

* Does this capability **modify choice** (before) or **log/guard** (after)?
* Is it **pure** (no I/O, no hidden globals)?
* Does it rely on a **clear, minimal** tool result shape?
* Could it run twice without breaking things (idempotent)?
* Could I **turn it off** without breaking tools?

# Micro-exercises (2–3 minutes each)

* **Redirect test:** Pretend the model picks `update_progress` first. Does PlanFirst return `("create_plan", {})`?
* **Result contract:** Change `update_progress` to return `{"progress_entry": ...}`—can you update the capability to handle either key?
* **Order swap:** What happens if ProgressTracking runs before PlanFirst at `before_tool`? (Nothing—good! You picked the right hooks.)
* **Duplicate safety:** Add a tiny check to avoid logging the *same* entry twice (e.g., compare last entry).

If these ideas feel clear, you’ve basically “got” capabilities: small, composable **policies** that use **hooks** to guide the agent without tangling up your tools.


# Final Code

In [ ]:
#===================
# Tool Registry
#===================

TOOL_REGISTRY: dict[str, callable] = {}

def register_tool(name: str | None = None):
    """Decorator: auto-register a function as a tool."""
    def deco(func):
        TOOL_REGISTRY[name or func.__name__] = func
        return func
    return deco

# (optional) helpers
def list_tools() -> list[str]: return sorted(TOOL_REGISTRY.keys())
def get_tool(name: str): return TOOL_REGISTRY[name]

#==================
# ActionContext
#==================

from dataclasses import dataclass, field
from typing import Any, Dict, Optional

@dataclass
class ActionContext:
    """
    Per-run backpack for tools.
    - memory: short-term data tools share this run (e.g., goals, progress)
    - deps: injected services/utilities (e.g., clock, clients, tokens)
    - config: knobs/flags for this run (e.g., max_iterations)
    """
    memory: Dict[str, Any] = field(default_factory=dict)
    deps: Dict[str, Any] = field(default_factory=dict)
    config: Dict[str, Any] = field(default_factory=dict)

    # (optional) tiny helpers
    def get_dep(self, name: str, default: Optional[Any] = None) -> Any:
        return self.deps.get(name, default)

    def remember(self, key: str, value: Any) -> None:
        self.memory[key] = value

#==========
# Tools
#==========

@register_tool()  # name defaults to "set_goals"
def set_goals(action_context: ActionContext, goals: list[str]) -> dict:
    # would write: action_context.memory["goals"] = list(goals)
    return {"goals": goals}

@register_tool()  # defaults to "create_plan"
def create_plan(action_context: ActionContext) -> dict:
    # would read: action_context.memory["goals"]
    # agent loop will store into: state["current_plan"]
    return {"plan": ["Step 1 ...", "Step 2 ...", "Step 3 ..."]}

@register_tool()  # defaults to "update_progress"
def update_progress(action_context: ActionContext,
                    step_index: int,
                    status: str,
                    note: str = "") -> dict:
    # would append to: action_context.memory["progress"]
    return {"ok": True, "entry": {"step": step_index, "status": status, "note": note}}


@register_tool(name="summarize_progress")  # explicit name (optional)
def summarize_progress(action_context: ActionContext) -> dict:
    # would read: state["current_plan"] + action_context.memory["progress"]
    return {"summary": "1 in progress, 2 remaining..."}


#==============
# Capability
#==============

class PlanFirstCapability:
    # HOOK: runs before any tool executes
    def before_tool(self, state, tool, args):
        if "current_plan" not in state and tool != "create_plan":
            # redirect to create_plan with no args (matches tool signature)
            return "create_plan", {}
        return tool, args

    def after_tool(self, state, tool, result):
        pass  # no-op

class ProgressTrackingCapability:
    def before_tool(self, state, tool, args):
        return tool, args  # no-op

    # HOOK: runs after a tool completes
    def after_tool(self, state, tool, result):
        if tool == "update_progress" and result.get("ok", True):
            entry = result.get("entry")
            if entry:
                state.setdefault("progress", []).append(entry)